# Using LLMChain
The LLMChain is most basic building block chain. It takes in a prompt template, formats it with the user input and returns the response from an LLM.

To use the LLMChain, first create a prompt template. 

In [3]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0, openai_api_key="OpenAPIkey")
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM.

In [4]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))



Socktastic!


If there are multiple variables, you can input them all at once using a dictionary.



In [5]:
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="What is a good name for {company} that makes {product}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))



Socktastic!


You can use a chat model in an LLMChain as well:

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0, openai_api_key="OpenAPIkey")
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

Rainbow Socks Co.


# Math chain
This notebook showcases using LLMs and Python REPLs to do complex word math problems.

In [8]:
from langchain import OpenAI, LLMMathChain

llm = OpenAI(temperature=0,  openai_api_key="OpenAPIkey")
llm_math = LLMMathChain.from_llm(llm, verbose=True)

llm_math.run("What is 13 raised to the .3432 power?")



> Entering new  chain...
What is 13 raised to the .3432 power?
```text
13**.3432
```
...numexpr.evaluate("13**.3432")...

Answer: 2.4116004626599237
> Finished chain.


'Answer: 2.4116004626599237'

# Bash chain
This notebook showcases using LLMs and a bash process to perform simple filesystem commands.

In [9]:
from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0,openai_api_key="OpenAPIkey")

text = "Please write a bash script that prints 'Hello World' to the console."

bash_chain = LLMBashChain.from_llm(llm, verbose=True)

bash_chain.run(text)



> Entering new  chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
echo "Hello World"
```
Code: ['echo "Hello World"']
Answer: "Hello World"

> Finished chain.


'"Hello World"\r\n'

# Router
This notebook demonstrates how to use the RouterChain paradigm to create a chain that dynamically selects the next chain to use for a given input.

Router chains are made up of two components:

The RouterChain itself (responsible for selecting the next chain to call)
destination_chains: chains that the router chain can route to
In this notebook we will focus on the different types of routing chains. We will show these routing chains used in a MultiPromptChain to create a question-answering chain that selects the prompt which is most relevant for a given question, and then answers the question using that prompt.

In [10]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [11]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [12]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [14]:
llm = OpenAI(temperature=0,openai_api_key="OpenAPIkey")
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [15]:
print(chain.run("What is gravity?"))



Gravity is a force of attraction between two objects that is proportional to their masses and inversely proportional to the square of the distance between them. It is the force that causes objects to fall to the ground when dropped and is the force that keeps the planets in orbit around the sun.


In [16]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"
    )
)

?

The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43.
